In [2]:
import os
import glob
from keras.preprocessing.image import load_img, img_to_array
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import KFold
import tensorflow as tf
from keras import layers, Model, Input
from keras.models import load_model

folder_path = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\Pupil\crop_img"
results = pd.read_excel("results.xlsx", header=None)

files = glob.glob(os.path.join(folder_path, '*.jpg'))

right_images = [file for file in files if 'right' in os.path.basename(file).lower()]
left_images = [file for file in files if 'left' in os.path.basename(file).lower()]
4
def load_and_preprocess_image(image_path, target_size=(64, 6)):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

right_images = np.array([load_and_preprocess_image(img_path) for img_path in right_images])
left_images = np.array([load_and_preprocess_image(img_path) for img_path in left_images])

labels = results[0].values
num_classes = len(np.unique(labels))
labels = to_categorical(labels, num_classes=num_classes)

def create_cnn_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Conv2D(16, kernel_size=(7, 7), activation='mish', padding='same', strides=(2, 2))(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Conv2D(16, kernel_size=(5, 5), activation='mish', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Conv2D(32, kernel_size=(3, 3), activation='mish', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Conv2D(64, kernel_size=(3, 3), activation='mish', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.5)(x)    

    x = layers.Conv2D(64, kernel_size=(3, 3), activation='mish', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Conv2D(128, kernel_size=(2, 2), activation='mish', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.5)(x)
    
    return Model(inputs=input_layer, outputs=x)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_train_accuracies = []
fold_val_accuracies = []

# 모델 저장
model_save_folder = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\models"
if not os.path.exists(model_save_folder):
    os.makedirs(model_save_folder)

# csv파일로 저장
csv_save_folder = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\predictions"
if not os.path.exists(csv_save_folder):
    os.makedirs(csv_save_folder)

for fold_idx, (train_index, val_index) in enumerate(kf.split(right_images)):
    right_images_train, right_images_val = right_images[train_index], right_images[val_index]
    left_images_train, left_images_val = left_images[train_index], left_images[val_index]
    labels_train, labels_val = labels[train_index], labels[val_index]

    cnn_right = create_cnn_model((64, 64, 3))
    cnn_left = create_cnn_model((64, 64, 3))

    # Flatten layers
    right_flatten = layers.Flatten()(cnn_right.output)
    left_flatten = layers.Flatten()(cnn_left.output)

    # Concatenate
    combined = layers.Concatenate()([right_flatten, left_flatten])

    fc_output = layers.Dense(128, activation='mish')(combined)
    fc_output = layers.BatchNormalization()(fc_output)
    fc_output = layers.Dropout(0.5)(fc_output)
    
    final_output = layers.Dense(num_classes, activation='softmax')(fc_output)

    model = Model(inputs=[cnn_right.input, cnn_left.input], outputs=final_output)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit([right_images_train, left_images_train], labels_train,
                        epochs=300,
                        batch_size=1, 
                        validation_data=([right_images_val, left_images_val], labels_val))

    fold_train_accuracies.append(history.history['accuracy'][-1])
    fold_val_accuracies.append(history.history['val_accuracy'][-1])

    print(f"Fold {fold_idx+1} - Train Accuracy: {fold_train_accuracies[-1]}, Validation Accuracy: {fold_val_accuracies[-1]}")

    model_save_path = os.path.join(model_save_folder, f"fold_{fold_idx+1}_model.h5")
    model.save(model_save_path)
    print(f"Fold {fold_idx+1} 모델 저장됨: {model_save_path}")

    predictions = model.predict([right_images_val, left_images_val])
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(labels_val, axis=1)

    right_image_files_val = [os.path.basename(files[idx]) for idx in val_index]
    left_image_files_val = [os.path.basename(files[idx]) for idx in val_index]

    # CSV에 저장할 데이터
    data = {
        'Right Image': right_image_files_val,
        'Left Image': left_image_files_val,
        'Actual Class': actual_classes,
        'Predicted Class': predicted_classes
    }
    df = pd.DataFrame(data)

    # CSV 파일 저장
    csv_save_path = os.path.join(csv_save_folder, f"fold_{fold_idx+1}_predictions.csv")
    df.to_csv(csv_save_path, index=False)
    print(f"Fold {fold_idx+1} 예측 결과 저장됨: {csv_save_path}")

avg_train_accuracy = np.mean(fold_train_accuracies)
avg_val_accuracy = np.mean(fold_val_accuracies)

print(f'Average Train Accuracy: {avg_train_accuracy}')
print(f'Average Validation Accuracy: {avg_val_accuracy}')


Epoch 1/300


ValueError: Input 0 of layer "functional_5" is incompatible with the layer: expected shape=(None, 64, 64, 3), found shape=(1, 64, 6, 3)

# VGG19

# cunstom vgg19

In [25]:
import os
import glob
import pandas as pd
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
from sklearn.model_selection import KFold
from keras import layers, Model
import tensorflow as tf

def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))

# 데이터 로드 경로
folder_path = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\Pupil\crop_img"
results = pd.read_excel("results.xlsx", header=None)

files = glob.glob(os.path.join(folder_path, '*.jpg'))

right_images = [file for file in files if 'right' in os.path.basename(file).lower()]
left_images = [file for file in files if 'left' in os.path.basename(file).lower()]

# 이미지 전처리 함수
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

# 이미지 배열로 변환
right_images = np.array([load_and_preprocess_image(img_path) for img_path in right_images])
left_images = np.array([load_and_preprocess_image(img_path) for img_path in left_images])

# 레이블 처리
labels = results[0].values
num_classes = len(np.unique(labels))
labels = to_categorical(labels, num_classes=num_classes)

# 커스텀 VGG19 모델 생성
def create_custom_vgg_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Convolutional Block 1
    x = layers.Conv2D(64, (3, 3), padding='same', activation=mish)(inputs)
    x = layers.Conv2D(64, (3, 3), padding='same', activation=mish)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional Block 2
    x = layers.Conv2D(128, (3, 3), padding='same', activation=mish)(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation=mish)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional Block 3
    x = layers.Conv2D(256, (3, 3), padding='same', activation=mish)(x)
    x = layers.Conv2D(256, (3, 3), padding='same', activation=mish)(x)
    x = layers.Conv2D(256, (3, 3), padding='same', activation=mish)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional Block 4
    x = layers.Conv2D(512, (3, 3), padding='same', activation=mish)(x)
    x = layers.Conv2D(512, (3, 3), padding='same', activation=mish)(x)
    x = layers.Conv2D(512, (3, 3), padding='same', activation=mish)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional Block 5
    x = layers.Conv2D(512, (3, 3), padding='same', activation=mish)(x)
    x = layers.Conv2D(512, (3, 3), padding='same', activation=mish)(x)
    x = layers.Conv2D(512, (3, 3), padding='same', activation=mish)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Flatten()(x)
    return Model(inputs, x)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_train_accuracies = []
fold_val_accuracies = []

# 모델 저장 경로
model_save_folder = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\models"
if not os.path.exists(model_save_folder):
    os.makedirs(model_save_folder)

# CSV 파일 저장 경로
csv_save_folder = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\predictions"
if not os.path.exists(csv_save_folder):
    os.makedirs(csv_save_folder)

for fold_idx, (train_index, val_index) in enumerate(kf.split(right_images)):
    right_images_train, right_images_val = right_images[train_index], right_images[val_index]
    left_images_train, left_images_val = left_images[train_index], left_images[val_index]
    labels_train, labels_val = labels[train_index], labels[val_index]

    cnn_right = create_custom_vgg_model((128, 128, 3))
    cnn_left = create_custom_vgg_model((128, 128, 3))

    # Flatten layers
    right_flatten = layers.Flatten()(cnn_right.output)
    left_flatten = layers.Flatten()(cnn_left.output)

    # Concatenate
    combined = layers.Concatenate()([right_flatten, left_flatten])

    fc_output = layers.Dense(128, activation=mish)(combined)
    fc_output = layers.BatchNormalization()(fc_output)
    fc_output = layers.Dropout(0.5)(fc_output)
    
    final_output = layers.Dense(num_classes, activation='softmax')(fc_output)

    model = Model(inputs=[cnn_right.input, cnn_left.input], outputs=final_output)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit([right_images_train, left_images_train], labels_train,
                        epochs=10,
                        batch_size=4, 
                        validation_data=([right_images_val, left_images_val], labels_val))

    fold_train_accuracies.append(history.history['accuracy'][-1])
    fold_val_accuracies.append(history.history['val_accuracy'][-1])

    print(f"Fold {fold_idx+1} - Train Accuracy: {fold_train_accuracies[-1]}, Validation Accuracy: {fold_val_accuracies[-1]}")

    model_save_path = os.path.join(model_save_folder, f"fold_{fold_idx+1}_model.h5")
    model.save(model_save_path)
    print(f"Fold {fold_idx+1} 모델 저장됨: {model_save_path}")

    predictions = model.predict([right_images_val, left_images_val])
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(labels_val, axis=1)

    right_image_files_val = [os.path.basename(files[idx]) for idx in val_index]
    left_image_files_val = [os.path.basename(files[idx]) for idx in val_index]

    # CSV에 저장할 데이터
    data = {
        'Right Image': right_image_files_val,
        'Left Image': left_image_files_val,
        'Actual Class': actual_classes,
        'Predicted Class': predicted_classes
    }
    df = pd.DataFrame(data)

    # CSV 파일 저장
    csv_save_path = os.path.join(csv_save_folder, f"fold_{fold_idx+1}_predictions.csv")
    df.to_csv(csv_save_path, index=False)
    print(f"Fold {fold_idx+1} 예측 결과 저장됨: {csv_save_path}")

avg_train_accuracy = np.mean(fold_train_accuracies)
avg_val_accuracy = np.mean(fold_val_accuracies)

print(f'Average Train Accuracy: {avg_train_accuracy}')
print(f'Average Validation Accuracy: {avg_val_accuracy}')


Epoch 1/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 619s 878ms/step - accuracy: 0.5993 - loss: 1.0255 - val_accuracy: 0.2500 - val_loss: 6.1408
Epoch 2/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 613s 877ms/step - accuracy: 0.7734 - loss: 0.6596 - val_accuracy: 0.2486 - val_loss: 2.7736
Epoch 3/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 613s 877ms/step - accuracy: 0.7910 - loss: 0.6061 - val_accuracy: 0.1671 - val_loss: 9.5234
Epoch 4/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 611s 875ms/step - accuracy: 0.8007 - loss: 0.6068 - val_accuracy: 0.7214 - val_loss: 0.7792
Epoch 5/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 616s 881ms/step - accuracy: 0.8071 - loss: 0.5782 - val_accuracy: 0.7314 - val_loss: 0.5815
Epoch 6/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 609s 872ms/step - accuracy: 0.8343 - loss: 0.5113 - val_accuracy: 0.6600 - val_loss: 1.1153
Epoch 7/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 609s 871ms/step - accuracy: 0.8573 - loss: 0.4591 - val_accuracy: 0.9457 - val_loss: 0.1813
Epoch 8/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 611s 875ms/step - accuracy: 0.8325 -

Fold 1 - Train Accuracy: 0.8594420552253723, Validation Accuracy: 0.30571427941322327
Fold 1 모델 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\models\fold_1_model.h5
22/22 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step
Fold 1 예측 결과 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\predictions\fold_1_predictions.csv
Epoch 1/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 636s 898ms/step - accuracy: 0.6405 - loss: 0.9865 - val_accuracy: 0.3305 - val_loss: 1.6126
Epoch 2/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 618s 883ms/step - accuracy: 0.7755 - loss: 0.7030 - val_accuracy: 0.2189 - val_loss: 6.7486
Epoch 3/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 624s 891ms/step - accuracy: 0.8232 - loss: 0.5505 - val_accuracy: 0.1831 - val_loss: 18.9759
Epoch 4/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 618s 883ms/step - accuracy: 0.8396 - loss: 0.5002 - val_accuracy: 0.4034 - val_loss: 1.3671
Epoch 5/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 617s 881ms/step - accuracy: 0.8459 - loss: 0.5068 - val_accuracy: 0.9585 - val_loss: 0.1236
Epoch 6/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 622

Fold 2 - Train Accuracy: 0.8852341771125793, Validation Accuracy: 0.9055793881416321
Fold 2 모델 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\models\fold_2_model.h5
22/22 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step
Fold 2 예측 결과 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\predictions\fold_2_predictions.csv
Epoch 1/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 610s 863ms/step - accuracy: 0.6381 - loss: 0.9635 - val_accuracy: 0.1860 - val_loss: 28.0752
Epoch 2/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 606s 865ms/step - accuracy: 0.7570 - loss: 0.6906 - val_accuracy: 0.5279 - val_loss: 3.0494
Epoch 3/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 605s 864ms/step - accuracy: 0.7916 - loss: 0.5990 - val_accuracy: 0.8541 - val_loss: 0.4176
Epoch 4/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 604s 863ms/step - accuracy: 0.8084 - loss: 0.5985 - val_accuracy: 0.3333 - val_loss: 3.0312
Epoch 5/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 602s 861ms/step - accuracy: 0.8148 - loss: 0.5834 - val_accuracy: 0.5379 - val_loss: 1.7293
Epoch 6/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 604s

Fold 3 - Train Accuracy: 0.8076510429382324, Validation Accuracy: 0.8297567963600159
Fold 3 모델 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\models\fold_3_model.h5
22/22 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step
Fold 3 예측 결과 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\predictions\fold_3_predictions.csv
Epoch 1/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 608s 860ms/step - accuracy: 0.6277 - loss: 0.9952 - val_accuracy: 0.4120 - val_loss: 1.9106
Epoch 2/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 608s 869ms/step - accuracy: 0.7871 - loss: 0.6178 - val_accuracy: 0.2132 - val_loss: 47.6990
Epoch 3/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 613s 876ms/step - accuracy: 0.7995 - loss: 0.5981 - val_accuracy: 0.2132 - val_loss: 14.5215
Epoch 4/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 616s 880ms/step - accuracy: 0.8428 - loss: 0.5446 - val_accuracy: 0.2475 - val_loss: 11.2219
Epoch 5/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 620s 886ms/step - accuracy: 0.8328 - loss: 0.5479 - val_accuracy: 0.2132 - val_loss: 24.9965
Epoch 6/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 6

Fold 4 - Train Accuracy: 0.8562746047973633, Validation Accuracy: 0.7525035738945007
Fold 4 모델 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\models\fold_4_model.h5
22/22 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step
Fold 4 예측 결과 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\predictions\fold_4_predictions.csv
Epoch 1/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 653s 923ms/step - accuracy: 0.6360 - loss: 0.9928 - val_accuracy: 0.1803 - val_loss: 9.6373
Epoch 2/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 627s 896ms/step - accuracy: 0.7861 - loss: 0.6364 - val_accuracy: 0.1803 - val_loss: 32.1241
Epoch 3/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 630s 900ms/step - accuracy: 0.8244 - loss: 0.5612 - val_accuracy: 0.1803 - val_loss: 26.7783
Epoch 4/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 625s 893ms/step - accuracy: 0.8109 - loss: 0.5811 - val_accuracy: 0.3476 - val_loss: 6.8653
Epoch 5/10
700/700 ━━━━━━━━━━━━━━━━━━━━ 628s 897ms/step - accuracy: 0.8559 - loss: 0.4523 - val_accuracy: 0.4492 - val_loss: 17786509312.0000
Epoch 6/10
700/700 ━━━━━━━━━━━━━━

Fold 5 - Train Accuracy: 0.8752234578132629, Validation Accuracy: 0.18025751411914825
Fold 5 모델 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\models\fold_5_model.h5
22/22 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step
Fold 5 예측 결과 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\predictions\fold_5_predictions.csv
Average Train Accuracy: 0.856765067577362
Average Validation Accuracy: 0.594762310385704


# 64X64

In [30]:
import os
import glob
import pandas as pd
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
from sklearn.model_selection import KFold
from keras import layers, Model
import tensorflow as tf

def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))

# 데이터 로드 경로
folder_path = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\Pupil\crop_img"
results = pd.read_excel("results.xlsx", header=None)

files = glob.glob(os.path.join(folder_path, '*.jpg'))

right_images = [file for file in files if 'right' in os.path.basename(file).lower()]
left_images = [file for file in files if 'left' in os.path.basename(file).lower()]

# 이미지 전처리 함수
def load_and_preprocess_image(image_path, target_size=(64, 64)):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

# 이미지 배열로 변환
right_images = np.array([load_and_preprocess_image(img_path) for img_path in right_images])
left_images = np.array([load_and_preprocess_image(img_path) for img_path in left_images])

# 레이블 처리
labels = results[0].values
num_classes = len(np.unique(labels))
labels = to_categorical(labels, num_classes=num_classes)

# 커스텀 VGG19 모델 생성
def create_custom_vgg_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Convolutional Block 1
    x = layers.Conv2D(64, (3, 3), padding='same', activation=mish)(inputs)
    x = layers.Conv2D(64, (3, 3), padding='same', activation=mish)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional Block 2
    x = layers.Conv2D(128, (3, 3), padding='same', activation=mish)(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation=mish)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional Block 3
    x = layers.Conv2D(256, (3, 3), padding='same', activation=mish)(x)
    x = layers.Conv2D(256, (3, 3), padding='same', activation=mish)(x)
    x = layers.Conv2D(256, (3, 3), padding='same', activation=mish)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional Block 4
    x = layers.Conv2D(512, (3, 3), padding='same', activation=mish)(x)
    x = layers.Conv2D(512, (3, 3), padding='same', activation=mish)(x)
    x = layers.Conv2D(512, (3, 3), padding='same', activation=mish)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional Block 5
    x = layers.Conv2D(512, (3, 3), padding='same', activation=mish)(x)
    x = layers.Conv2D(512, (3, 3), padding='same', activation=mish)(x)
    x = layers.Conv2D(512, (3, 3), padding='same', activation=mish)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Flatten()(x)
    return Model(inputs, x)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_train_accuracies = []
fold_val_accuracies = []

# 모델 저장 경로
model_save_folder = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\vgg_models"
if not os.path.exists(model_save_folder):
    os.makedirs(model_save_folder)

# CSV 파일 저장 경로
csv_save_folder = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\vgg_predictions"
if not os.path.exists(csv_save_folder):
    os.makedirs(csv_save_folder)

for fold_idx, (train_index, val_index) in enumerate(kf.split(right_images)):
    right_images_train, right_images_val = right_images[train_index], right_images[val_index]
    left_images_train, left_images_val = left_images[train_index], left_images[val_index]
    labels_train, labels_val = labels[train_index], labels[val_index]

    cnn_right = create_custom_vgg_model((64, 64, 3))
    cnn_left = create_custom_vgg_model((64, 64, 3))

    # Flatten layers
    right_flatten = layers.Flatten()(cnn_right.output)
    left_flatten = layers.Flatten()(cnn_left.output)

    # Concatenate
    combined = layers.Concatenate()([right_flatten, left_flatten])

    fc_output = layers.Dense(1024, activation=mish)(combined)
    fc_output = layers.Dense(512, activation=mish)(fc_output)
    fc_output = layers.Dense(256, activation=mish)(fc_output)
    fc_output = layers.Dense(128, activation=mish)(fc_output)
    fc_output = layers.BatchNormalization()(fc_output)
    fc_output = layers.Dropout(0.5)(fc_output)
    
    final_output = layers.Dense(num_classes, activation='softmax')(fc_output)

    model = Model(inputs=[cnn_right.input, cnn_left.input], outputs=final_output)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit([right_images_train, left_images_train], labels_train,
                        epochs=100,
                        batch_size=8, 
                        validation_data=([right_images_val, left_images_val], labels_val))

    fold_train_accuracies.append(history.history['accuracy'][-1])
    fold_val_accuracies.append(history.history['val_accuracy'][-1])

    print(f"Fold {fold_idx+1} - Train Accuracy: {fold_train_accuracies[-1]}, Validation Accuracy: {fold_val_accuracies[-1]}")

    model_save_path = os.path.join(model_save_folder, f"fold_{fold_idx+1}_model.h5")
    model.save(model_save_path)
    print(f"Fold {fold_idx+1} 모델 저장됨: {model_save_path}")

    predictions = model.predict([right_images_val, left_images_val])
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(labels_val, axis=1)

    right_image_files_val = [os.path.basename(files[idx]) for idx in val_index]
    left_image_files_val = [os.path.basename(files[idx]) for idx in val_index]

    # CSV에 저장할 데이터
    data = {
        'Right Image': right_image_files_val,
        'Left Image': left_image_files_val,
        'Actual Class': actual_classes,
        'Predicted Class': predicted_classes
    }
    df = pd.DataFrame(data)

    # CSV 파일 저장
    csv_save_path = os.path.join(csv_save_folder, f"fold_{fold_idx+1}_predictions.csv")
    df.to_csv(csv_save_path, index=False)
    print(f"Fold {fold_idx+1} 예측 결과 저장됨: {csv_save_path}")

avg_train_accuracy = np.mean(fold_train_accuracies)
avg_val_accuracy = np.mean(fold_val_accuracies)

print(f'Average Train Accuracy: {avg_train_accuracy}')
print(f'Average Validation Accuracy: {avg_val_accuracy}')


Epoch 1/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 237s 660ms/step - accuracy: 0.3780 - loss: 1.4383 - val_accuracy: 0.2943 - val_loss: 9.0208
Epoch 2/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 223s 636ms/step - accuracy: 0.4545 - loss: 1.1563 - val_accuracy: 0.2386 - val_loss: 8.4108
Epoch 3/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 223s 637ms/step - accuracy: 0.5463 - loss: 1.0236 - val_accuracy: 0.2386 - val_loss: 30.0297
Epoch 4/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 229s 655ms/step - accuracy: 0.6060 - loss: 0.9094 - val_accuracy: 0.1671 - val_loss: 10.7762
Epoch 5/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 227s 648ms/step - accuracy: 0.7797 - loss: 0.6283 - val_accuracy: 0.2386 - val_loss: 8.8749
Epoch 6/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 232s 664ms/step - accuracy: 0.8314 - loss: 0.5226 - val_accuracy: 0.4386 - val_loss: 2.9719
Epoch 7/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 227s 650ms/step - accuracy: 0.8421 - loss: 0.4629 - val_accuracy: 0.3343 - val_loss: 2.8171
Epoch 8/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 231s 659ms/step - accuracy

Fold 1 - Train Accuracy: 0.19742488861083984, Validation Accuracy: 0.167142853140831
Fold 1 모델 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\vgg_models\fold_1_model.h5
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 333ms/step
Fold 1 예측 결과 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\vgg_predictions\fold_1_predictions.csv
Epoch 1/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 235s 653ms/step - accuracy: 0.4304 - loss: 1.3295 - val_accuracy: 0.1831 - val_loss: 16.1223
Epoch 2/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 231s 659ms/step - accuracy: 0.6497 - loss: 0.7870 - val_accuracy: 0.2146 - val_loss: 12.3826
Epoch 3/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 234s 667ms/step - accuracy: 0.7732 - loss: 0.6713 - val_accuracy: 0.1831 - val_loss: 15.0240
Epoch 4/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 233s 665ms/step - accuracy: 0.8068 - loss: 0.5215 - val_accuracy: 0.1831 - val_loss: 19.1490
Epoch 5/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 233s 665ms/step - accuracy: 0.8176 - loss: 0.5256 - val_accuracy: 0.2847 - val_loss: 6.4382
Epoch 6/100
350/350 ━━━━━━

Fold 2 - Train Accuracy: 0.19377905130386353, Validation Accuracy: 0.18884120881557465
Fold 2 모델 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\vgg_models\fold_2_model.h5
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 334ms/step
Fold 2 예측 결과 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\vgg_predictions\fold_2_predictions.csv
Epoch 1/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 228s 628ms/step - accuracy: 0.4561 - loss: 1.2682 - val_accuracy: 0.2003 - val_loss: 58.5103
Epoch 2/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 222s 633ms/step - accuracy: 0.6699 - loss: 0.7996 - val_accuracy: 0.1860 - val_loss: 34.5695
Epoch 3/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 219s 627ms/step - accuracy: 0.8118 - loss: 0.5388 - val_accuracy: 0.1874 - val_loss: 13.9908
Epoch 4/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 221s 630ms/step - accuracy: 0.8446 - loss: 0.4441 - val_accuracy: 0.2518 - val_loss: 11.4579
Epoch 5/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 223s 636ms/step - accuracy: 0.8387 - loss: 0.4640 - val_accuracy: 0.1874 - val_loss: 16.0567
Epoch 6/100
350/350 ━━━

Fold 3 - Train Accuracy: 0.1941365748643875, Validation Accuracy: 0.19170242547988892
Fold 3 모델 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\vgg_models\fold_3_model.h5
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 334ms/step
Fold 3 예측 결과 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\vgg_predictions\fold_3_predictions.csv
Epoch 1/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 225s 624ms/step - accuracy: 0.3936 - loss: 1.4606 - val_accuracy: 0.2175 - val_loss: 7.2987
Epoch 2/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 218s 623ms/step - accuracy: 0.5285 - loss: 1.0996 - val_accuracy: 0.2132 - val_loss: 40.1179
Epoch 3/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 216s 617ms/step - accuracy: 0.7318 - loss: 0.7186 - val_accuracy: 0.2132 - val_loss: 49.7194
Epoch 4/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 216s 616ms/step - accuracy: 0.8046 - loss: 0.5791 - val_accuracy: 0.2690 - val_loss: 3.5128
Epoch 5/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 216s 618ms/step - accuracy: 0.8885 - loss: 0.3429 - val_accuracy: 0.2132 - val_loss: 9.3578
Epoch 6/100
350/350 ━━━━━━━

Fold 4 - Train Accuracy: 0.21558813750743866, Validation Accuracy: 0.18454936146736145
Fold 4 모델 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\vgg_models\fold_4_model.h5
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 334ms/step
Fold 4 예측 결과 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\vgg_predictions\fold_4_predictions.csv
Epoch 1/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 223s 619ms/step - accuracy: 0.4624 - loss: 1.3053 - val_accuracy: 0.1803 - val_loss: 24.7654
Epoch 2/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 218s 623ms/step - accuracy: 0.6899 - loss: 0.8051 - val_accuracy: 0.2089 - val_loss: 8016137216.0000
Epoch 3/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 218s 624ms/step - accuracy: 0.7315 - loss: 0.7080 - val_accuracy: 0.2089 - val_loss: 50.6185
Epoch 4/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 218s 623ms/step - accuracy: 0.8415 - loss: 0.5043 - val_accuracy: 0.1788 - val_loss: 36.7182
Epoch 5/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 218s 623ms/step - accuracy: 0.8551 - loss: 0.4165 - val_accuracy: 0.1803 - val_loss: 36.0820
Epoch 6/100
350

Fold 5 - Train Accuracy: 0.2095101922750473, Validation Accuracy: 0.18025751411914825
Fold 5 모델 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\vgg_models\fold_5_model.h5
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 338ms/step
Fold 5 예측 결과 저장됨: C:\Users\IMS\Desktop\Hwangsihoon\WebCam\vgg_predictions\fold_5_predictions.csv
Average Train Accuracy: 0.20208776891231536
Average Validation Accuracy: 0.18249867260456085


In [32]:
model.summary()

Model: "functional_74"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_59 (InputLayer)   │ (None, 64, 64, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_60 (InputLayer)   │ (None, 64, 64, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_492 (Conv2D)           │ (None, 64, 64, 64)        │           1,792 │ input_layer_59[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_505 (Conv2D)           │ (None, 64, 64, 64)        │           1,792 │ input_layer_60[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_493 (Conv2D)           │ (None, 64, 64, 64)        │          36,928 │ conv2d_492[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_506 (Conv2D)           │ (None, 64, 64, 64)        │          36,928 │ conv2d_505[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_204             │ (None, 32, 32, 64)        │               0 │ conv2d_493[0][0]           │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_209             │ (None, 32, 32, 64)        │               0 │ conv2d_506[0][0]           │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_494 (Conv2D)           │ (None, 32, 32, 128)       │          73,856 │ max_pooling2d_204[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_507 (Conv2D)           │ (None, 32, 32, 128)       │          73,856 │ max_pooling2d_209[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_495 (Conv2D)           │ (None, 32, 32, 128)       │         147,584 │ conv2d_494[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_508 (Conv2D)           │ (None, 32, 32, 128)       │         147,584 │ conv2d_507[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_205             │ (None, 16, 16, 128)       │               0 │ conv2d_495[0][0]           │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_210             │ (None, 16, 16, 128)       │               0 │ conv2d_508[0][0]           │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_496 (Conv2D)           │ (None, 16, 16, 256)       │         295,168 │ max_pooling2d_205[0][0]    │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 102,944,145 (392.70 MB)

 Trainable params: 34,314,629 (130.90 MB)

 Non-trainable params: 256 (1.00 KB)

 Optimizer params: 68,629,260 (261.80 MB)